Aqui, começaremos a atacar o problema de sincronizar nossos arquivos de audio com os acordes escritos. Primeiramente, tentaremos, para uma dada música, determinar os acordes por compasso. Faremos isso tomando como base os arquivos em formato xml.

In [36]:
import pandas as pd
import numpy as np
import os
import xml.etree.ElementTree as ET
tree = ET.parse('corpus/xml/Ballad.xml')
root = tree.getroot()

In [7]:
part = root.findall('part')[0]
measures = part.findall('measure')
for measure in measures:
    chords = measure.findall('harmony')
    for chord in chords:
        try:
            note = chord.find('root/root-step').text

        except AttributeError: # se não achar a nota "root" do acorde, não tem acorde
            print('Measure: {}'.format(measure.attrib['number']), ' | ', 'No chords')
            continue
            
        if note:
            # alter é a alteração de um acorde (nem sempre existe): b ou #.
            alter = chord.find('root/root-alter')#.text
            
            # kind é o tipo do acorde (nem sempre existe): pode ser com 7a, 9a, 13a, por aí vai...
            kind = chord.find('kind')#.attrib['text']
            
            # bass é o baixo do acorde, quando diferente da tônica. Também nem sempre existe.
            bass = chord.find('bass/bass-step')#.text
            
            # bass_alter é e alteração do baixo, podendo ser b ou #. Também nem sempre existe.
            bass_alter = chord.find('bass/bass-alter')#.text
            
            attribs_dict = {'alter':alter, 
                            'kind':kind, 
                            'bass':bass, 
                            'bass_alter':bass_alter}

            for key, value in attribs_dict.items():
                if value is None:
                    attribs_dict[key] = ''
                elif key == 'kind':
                    attribs_dict[key] = value.attrib['text']
                else:
                    attribs_dict[key] = value.text
                    
            print('Measure: {}'.format(measure.attrib['number']), '|', 'Chord: ', note, 
                  attribs_dict['alter'], attribs_dict['kind'],'/',attribs_dict['bass'],attribs_dict['bass_alter'])
            

Measure: 3 | Chord:  E -1 min7 /  
Measure: 3 | Chord:  D -1  /  
Measure: 4 | Chord:  D -1  /  
Measure: 4 | Chord:  A -1  /  
Measure: 5 | Chord:  E -1 min7 /  
Measure: 5 | Chord:  D -1  / E -1
Measure: 6 | Chord:  D -1  / E -1
Measure: 6 | Chord:  A -1  / E -1
Measure: 7 | Chord:  E -1 min7 /  
Measure: 7 | Chord:  D -1  /  
Measure: 8 | Chord:  B   / E -1
Measure: 8 | Chord:  D -1 7 /  
Measure: 9 | Chord:  G -1 min7 /  
Measure: 10 | Chord:  E  min7 /  
Measure: 10 | Chord:  F 1  /  
Measure: 11 | Chord:  B  min7 /  
Measure: 12 | Chord:  G -1  /  
Measure: 12 | Chord:  B  7 /  
Measure: 13 | Chord:  E  maj7 /  
Measure: 14 | Chord:  A -1  /  
Measure: 14 | Chord:  A -1  / B -1
Measure: 15 | Chord:  E -1 min7 /  
Measure: 15 | Chord:  D -1  /  
Measure: 16 | Chord:  F  min7 /  
Measure: 16 | Chord:  B -1 7 /  
Measure: 17 | Chord:  G  min7 /  
Measure: 18 | Chord:  A -1 min7 /  
Measure: 18 | Chord:  A -1  /  
Measure: 19 | Chord:  G -1 7 /  
Measure: 20 | Chord:  A -1  / B -1
Me

Comparando com a partitura aberta no musescore parece fazer sentido, só preciso conseguir fazer com que mostre também os compassos em que não tem acorde (no caso da música que estamos usando como exemplo, os compassos 1 e 2).

Agora, vamos modificar ligeiramente o código acima para adicionar as informações em um dataframe.

In [8]:
df_teste = pd.DataFrame(columns=['Ballad'])
df_add = pd.DataFrame(columns=['Ballad'], 
                      index=[3], 
                      data='E -1 min7')

df_add_2 = pd.DataFrame(columns=['Ballad'], 
                      index=[3], 
                      data='D -1')

In [9]:
df_teste = df_teste.append(df_add)

In [10]:
df_teste.append(df_add_2)

,Ballad
3,E -1 min7
3,D -1


In [11]:
song = 'Ballad'
df_harmony = pd.DataFrame(columns=[song])

part = root.findall('part')[0]
measures = part.findall('measure')

for measure in measures:
    chords = measure.findall('harmony')
    for chord in chords:
        try:
            note = chord.find('root/root-step').text

        except AttributeError: # se não achar a nota "root" do acorde, não tem acorde
            print('Measure: {}'.format(measure.attrib['number']), ' | ', 'No chords')
            continue
            
        if note:
            # alter é a alteração de um acorde (nem sempre existe): b ou #.
            alter = chord.find('root/root-alter')#.text
            
            # kind é o tipo do acorde (nem sempre existe): pode ser com 7a, 9a, 13a, por aí vai...
            kind = chord.find('kind')#.attrib['text']
            
            # bass é o baixo do acorde, quando diferente da tônica. Também nem sempre existe.
            bass = chord.find('bass/bass-step')#.text
            
            # bass_alter é e alteração do baixo, podendo ser b ou #. Também nem sempre existe.
            bass_alter = chord.find('bass/bass-alter')#.text
            
            attribs_dict = {'alter':alter, 
                            'kind':kind, 
                            'bass':bass, 
                            'bass_alter':bass_alter}

            for key, value in attribs_dict.items():
                if value is None:
                    attribs_dict[key] = ''
                elif key == 'kind':
                    attribs_dict[key] = value.attrib['text']
                else:
                    attribs_dict[key] = value.text
                    
                    
            measure_number = measure.attrib['number']
            chord = note + attribs_dict['alter'] + attribs_dict['kind']
            chord_bass = attribs_dict['bass']+attribs_dict['bass_alter']
            complete_chord = chord + '/' + chord_bass
            
            if complete_chord.endswith('/'):
                complete_chord = complete_chord[:-1]
            
            df_chord = pd.DataFrame(columns=[song], index=[measure_number], data=complete_chord)
            df_harmony = df_harmony.append(df_chord)
            
df_harmony.index.names = ['Measure']

In [27]:
df_harmony.index.unique()

Index(['3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15',
       '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27'],
      dtype='object', name='Measure')

In [29]:
df_harmony.index.value_counts()

21    3
20    3
12    2
22    2
7     2
16    2
5     2
4     2
10    2
18    2
3     2
6     2
8     2
15    2
14    2
26    1
27    1
23    1
11    1
13    1
17    1
25    1
24    1
19    1
9     1
Name: Measure, dtype: int64

In [28]:
df_harmony

,Ballad
Measure,
3,E-1min7
3,D-1
4,D-1
4,A-1
5,E-1min7
5,D-1/E-1
6,D-1/E-1
6,A-1/E-1
7,E-1min7


Agora vamos procurar os momentos de mudança de acorde. Vasculhando os arquivos xml que temos em nosso corpus, pudemos ver que as informações sobre andamento (em bpm) e tipo de compasso encontram-se na seção do primeiro compasso. Segue:

#### Tipo de compasso

In [51]:
first_measure = measures[0]
cima = first_measure.find('attributes').find('time/beats').text
baixo = first_measure.find('attributes').find('time/beat-type').text

compasso = cima + '/' + baixo
compasso

'4/4'

#### Andamento

In [52]:
andamento = first_measure.find('sound').attrib['tempo']# + ' bpm'
andamento
# em bpm

'120'

#### Sync: Acorde + minutagem

In [87]:
df_harmony['Time'] = ''
df_harmony

,Ballad,Time
Measure,,
3,E-1min7,
3,D-1,
4,D-1,
4,A-1,
5,E-1min7,
5,D-1/E-1,
6,D-1/E-1,
6,A-1/E-1,
7,E-1min7,


In [93]:
duracao_seminima = 60/int(andamento) # em segundos

Fazendo caso a caso pra construir a generalização:

In [98]:
# if int(baixo) == 2: # a unidade é a mínima
#     segs_compasso = 2 * int(cima) * duracao_seminima
    
# elif int(baixo) == 4: # a unidade é a seminínima
#     segs_compasso = 1 * int(cima) * duracao_seminima
    
# elif int(baixo) == 8: # a unidade é a colcheia
#     segs_compasso = (1/2) * int(cima) * duracao_seminima
    
# elif int(baixo) == 16: # a unidade é a semicolcheia
#     segs_compasso = (1/4) * int(cima) * duracao_seminima

Checar se a generalização abaixo tá certa:

In [94]:
segs_compasso = (4/int(baixo)) * int(cima) * duracao_seminima

In [95]:
df_harmony.index = df_harmony.index.astype(int)

In [96]:
for compasso in df_harmony.index.unique():
    acordes = df_harmony.loc[[compasso]]
    n_acordes = len(acordes)
    print(compasso, n_acordes)
    inicio_compasso = (int(compasso) - 1) * segs_compasso
    
    if n_acordes == 1:
        tempos = inicio_compasso
    
    else:        
        tempos = np.linspace(inicio_compasso,
                             inicio_compasso + segs_compasso,
                             n_acordes+1)
        tempos = tempos[:-1]
#     df_harmony.loc[compasso]['Time'] = tempos
    
    df_harmony.at[compasso, 'Time'] = tempos

3 2
4 2
5 2
6 2
7 2
8 2
9 1
10 2
11 1
12 2
13 1
14 2
15 2
16 2
17 1
18 2
19 1
20 3
21 3
22 2
23 1
24 1
25 1
26 1
27 1


In [97]:
df_harmony

,Ballad,Time
Measure,,
3,E-1min7,4
3,D-1,5
4,D-1,6
4,A-1,7
5,E-1min7,8
5,D-1/E-1,9
6,D-1/E-1,10
6,A-1/E-1,11
7,E-1min7,12


Botei pra tocar e as mudanças de acorde parecem estar fazendo sentido! 
Uma coisa que acabei de pensar é sobre os ritornellos... acho que desta maneira não os reconhecemos. Verificar depois.

Pra facilitar testes: músicas com compasso diferente de 4/4

In [20]:
# files = os.listdir('corpus/xml/')
# for file in files:
#     tree = ET.parse('corpus/xml/'+file)
#     root = tree.getroot()

#     part = root.findall('part')[0]
#     measures = part.findall('measure')
#     first_measure = measures[0]
#     cima = first_measure.find('attributes').find('time/beats').text
#     baixo = first_measure.find('attributes').find('time/beat-type').text

#     compasso = cima + '/' + baixo
    
#     if compasso != '4/4':
#         print(file, compasso)